In [1]:
# How did leetcode go?
# Any homework questions?

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Today we will learn linear regression.

# We are a team of fruit playing sports 
# Each team has a value for how much their sports team would cost to buy, 
# based on what fruit they have.

In [6]:
np.random.seed(0)
num_data = 30
bananas = np.random.randint(10, size=num_data)
strawberries = np.random.randint(10, size=num_data)
mangos = np.random.randint(10, size=num_data)

value_bananas = 1
value_strawberries = 2
value_mangos = .8

value_team = (bananas * value_bananas + 
              strawberries * value_strawberries + 
              mangos * value_mangos +
              np.random.rand(num_data) * 10 - 5)

df = pd.DataFrame(
    {'bananas':bananas,
     'strawberries':strawberries, 
     'mangos':mangos, 
     'value_team':value_team})

df.head(3)

,bananas,strawberries,mangos,value_team
0,5,2,8,18.183455
1,0,3,1,11.295711
2,3,8,1,21.425269


In [5]:
# The goal of Linear regression is to look at the noisy data from above and predict what is the
# Value of each individual fruit. Because we see the calculation, we know the right values,
# for instance

# value_bananas = 1
# value_strawberries = 2
# value_mangos = .8

# But in most cases we have the dataset about values but don't know the true value of each fruit.

In [6]:
# Here let's implement linear regression.
# Let's look at one row at a time.

In [7]:
pred_bananas = 1
pred_strawberries = 1
pred_mangos = 1

In [8]:
# Predict the value of the first row
prediction = (df.loc[0, 'bananas'] * pred_bananas + 
              df.loc[0, 'strawberries'] * pred_strawberries + 
              df.loc[0, 'mangos'] * pred_mangos)

In [9]:
# Is your prediction perfect? Would you rather it be bigger or smaller? 
# Calculate the "error" bewtween your the real value - prediction 

In [9]:
error = df.loc[0, 'value_team'] - prediction

error

3.18345482025909

In [11]:
# If we want the prediction to be bigger, what would be the most efficient way to change our
# predicted values of each fruit to achieve that?
# Well, since mangos is 8 and strawberries is 2, if we increase mangos that will have the biggest
# impact at resolving our error.
# So lets look at the loss and multiply it by the sample value to estimate how much to
# change our predicted multipliers
# (lets scale our change by a small fraction like a 'learning rate' of .001 so we don't over-correct)

In [10]:
lr = .001
pred_bananas = pred_bananas + error * df.loc[0, 'bananas'] * lr
pred_strawberries = pred_strawberries + error * df.loc[0, 'strawberries'] * lr
pred_mangos = pred_mangos + error * df.loc[0, 'mangos'] * lr

In [13]:
# Now let's check if our prediction has a smaller error

In [12]:
prediction = (df.loc[0, 'bananas'] * pred_bananas + 
              df.loc[0, 'strawberries'] * pred_strawberries + 
              df.loc[0, 'mangos'] * pred_mangos)

In [15]:
error = df.loc[0, 'value_team'] - prediction
error

2.887393521974996

In [16]:
# Wooo! The error went down. So let's think about it. Y = A * x.
# We are learning A by looking at the desired changes in Y, and how that effects desired changes in A
# We calculate that by DA = DY * DY/DA
# And remember, DY/DA = x for y = ax.
# And that's all the math that we will ever use from calculus.

In [17]:
# Now please use a numpy vector of 3 numbers instead of three separate variables
# To achieve the same results.

In [18]:
# predicted_multipliers
pred_mult = np.ones(3)

In [19]:
# make predictions with vectors
prediction = (pred_mult * df.iloc[0, :3]).sum()
error = df.loc[0, 'value_team'] - prediction
error

3.18345482025909

In [20]:
# update learned vector
pred_mult = pred_mult + error * df.iloc[0, :3] * lr

In [21]:
# check the updated error
prediction = (pred_mult * df.iloc[0, :3]).sum()
error = df.loc[0, 'value_team'] - prediction
error

2.887393521974996

In [22]:
# You shoud get the same value as before.
# Now let's show we can iterate over all the values in the dataset, performing this update.
# Eventually we should learn the correct values

In [23]:
for i in range(df.shape[0]):
    prediction = (pred_mult * df.iloc[i, :3]).sum()
    error = df.loc[i, 'value_team'] - prediction
    pred_mult = pred_mult + error * df.iloc[i, :3] * lr

In [24]:
pred_mult

bananas         1.283390
strawberries    1.429726
mangos          1.125828
dtype: float64

In [25]:
# We want to see 
# value_bananas = 1
# value_strawberries = 2
# value_mangos = .8
# So we are still a long way off.
# What if we called a loop through the dataset an "epoch" and we went through 10 epochs?

In [26]:
num_epochs = 10
for epoch in range(num_epochs):
    for i in range(df.shape[0]):
        prediction = (pred_mult * df.iloc[i, :3]).sum()
        error = df.loc[i, 'value_team'] - prediction
        pred_mult = pred_mult + error * df.iloc[i, :3] * lr


In [27]:
pred_mult

bananas         1.357265
strawberries    1.972966
mangos          0.607794
dtype: float64

In [28]:
# Pretty close! Lastly, let's do a mathematical hack.
# Rather than iterating over each of the datapoints, let's do it all at once.
# Optimizing over one data point at a time is called stochastic optimization. 
# Using the whole dataset (batch) at once is called batch optimization
# Using chunks of data is called mini-batch optimization.
# Let's implement batch optimization because it is fast.

In [29]:
pred_mult = np.ones(3)
# First, predict for all the datapoints.
x = df.iloc[:,:3].values

In [30]:
# How do we multiply a each row of a numpy array by a vector, getting a vector of outputs
# (one output per row)
# Hint: Use the dot product multiplier
yhat = x.dot(pred_mult)
yhat.shape

(30,)

In [31]:
# Now calculate the error for each row as a vector
error = yhat - df['value_team']

In [32]:
# now calculate the update for the parameters for each for
proposed_changes = np.zeros(3)
for i in range(len(error)):
    xi = x[i,:]
    proposed_changes += xi * error[i]

In [33]:
proposed_changes

array([-625.19558961, -745.97079025, -399.7725966 ])

In [34]:
# Now let's skip some linear algebra. What if we wanted to take the multiplication of a 30x1 matrix
# with a 30x3 matrix, and we wanted the output to be the sum over the 30 rows, of each of the
# row wise multiplication.
# Try to use a dot product to get a 3x1 shaped output from this multiplication
# Show that you get the same result as the proposed changes from above.

In [35]:
pred_change = error.dot(x)/len(error)
pred_change

array([-20.83985299, -24.86569301, -13.32575322])

In [36]:
# Great! This is exactly how modern math is done. You check what shapes of data you have, and you 
# know if you want to multiply or add them together, and you use dot products and check
# That the output is the shape you expect

In [37]:
# Now let's put everything together and run linear regression efficiently

In [38]:
pred_mult = np.ones(3)


In [39]:
num_epochs = 1000
for epoch in range(num_epochs):
    x = df.iloc[:,:3].values
    yhat = x.dot(pred_mult)
    error = yhat - df['value_team']
    pred_change = error.dot(x)/len(error)

    pred_mult = pred_mult - pred_change * lr
pred_mult

array([1.39387586, 2.00601779, 0.55046287])

In [40]:
# What could we do to make this more accurate?
# Can you functionize the data generation and model running to simulate a similar dataset, then 
# learn a model from it and check if the model learns the correct data distribution?